In [ ]:
import tensorflow as tf
import input_data
import os
import numpy as np
from tensorflow.python.framework import graph_util
from tensorflow.python.profiler import option_builder
from tensorflow.python.profiler import model_analyzer
#import SSDC_Densenet as model
import model_test_3d as model
#import fdssc as model

In [ ]:
def load_pb(pb):
    with tf.gfile.GFile(pb, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        # fix nodes
        for node in graph_def.node:
            if node.op == 'RefSwitch':
                node.op = 'Switch'
                for index in xrange(len(node.input)):
                    if 'moving_' in node.input[index]:
                        node.input[index] = node.input[index] + '/read'
            elif node.op == 'AssignSub':
                node.op = 'Sub'
                if 'use_locking' in node.attr: del node.attr['use_locking']

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

In [ ]:
def stats_graph(graph):

    # Count param
    param_opts = tf.profiler.ProfileOptionBuilder(tf.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    #param_opts.with_max_depth(4)
    param_opts.order_by('occurrence')
    param_stats = tf.profiler.profile(graph, options=param_opts.build(), cmd='scope')
    print('Trainable params: ', param_stats.total_parameters)
    for x in param_stats.children:
        print(x.name, 'scope trainable params: ', x.total_parameters)
    
    # Count FLOPs
    float_opts = tf.profiler.ProfileOptionBuilder(tf.profiler.ProfileOptionBuilder.float_operation())
    #float_opts.with_max_depth(4)
    #float_opts.order_by('name')
    float_stats = tf.profiler.profile(graph, options=float_opts.build(), cmd='op')
    print('Total FLOPs: ', float_stats.total_float_ops)
    for x in float_stats.children:
        print(x.name, 'scope FLOPs: ', x.total_float_ops)



In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
with tf.Graph().as_default() as graph:
    # ***** (1) Create Graph *****
#     A = tf.Variable(initial_value=tf.random_normal([25, 16]))
#     B = tf.Variable(initial_value=tf.random_normal([16, 9]))
#     C = tf.matmul(A, B, name='output')
    images = tf.Variable(initial_value = tf.random_normal([32,7*7*200]))
    output = model.inference(images, tf.cast(True, tf.bool))
    
    print('stats before freezing')
    stats_graph(graph)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # ***** (2) freeze graph *****
        output_graph = graph_util.convert_variables_to_constants(sess, graph.as_graph_def(), ['Block3/fc/fc/output']) #'fc/output' #'Block3/fc/fc/output'
        with tf.gfile.GFile('graph.pb', "wb") as f:
            f.write(output_graph.SerializeToString())

In [ ]:
# ***** (3) Load frozen graph *****
graph = load_pb('./graph.pb')
print('stats after freezing')
stats_graph(graph)